In [1]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import KFold
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score


%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}
pd.options.mode.chained_assignment = None  # default='warn'?

# data_key DataFrame
data_key = pd.read_csv('key.csv')

# data_train DataFrame
data_train = pd.read_csv('train.csv')

# data_weather DataFrame
data_weather = pd.read_csv('weather.csv')

rain_text = ['FC', 'TS', 'GR', 'RA', 'DZ', 'SN', 'SG', 'GS', 'PL', 'IC', 'FG', 'BR', 'UP', 'FG+']
other_text = ['HZ', 'FU', 'VA', 'DU', 'DS', 'PO', 'SA', 'SS', 'PY', 'SQ', 'DR', 'SH', 'FZ', 'MI', 'PR', 'BC', 'BL', 'VC' ]

data_weather['codesum'].replace("+", "")
a = []
for i in range(len(data_weather['codesum'])):
    a.append(data_weather['codesum'].values[i].split(" "))
    for i_text in a[i]:
        if len(i_text) == 4:
            a[i].append(i_text[:2])
            a[i].append(i_text[2:])
            
data_weather["nothing"] = 1
data_weather["rain"] = 0
data_weather["other"] = 0
b = -1
for ls in a:
    b += 1
    for text in ls:
        if text in rain_text:
            data_weather.loc[b, 'rain'] = 1
            data_weather.loc[b, 'nothing'] = 0
        elif text in other_text:
            data_weather.loc[b,'other'] = 1
            data_weather.loc[b, 'nothing'] = 0      
# 모든 데이터 Merge
df = pd.merge(data_weather, data_key)

station_nbr = df['station_nbr']
df.drop('station_nbr', axis=1, inplace=True)
df['station_nbr'] = station_nbr

df = pd.merge(df, data_train)

# T 값 처리 하기. Remained Subject = > 'M' and '-'
df['snowfall'][df['snowfall'] == '  T'] = 0.05
df['preciptotal'][df['preciptotal'] == '  T'] = 0.005

# 주말과 주중 구분 작업 하기
df['date'] = pd.to_datetime(df['date'])
df['week7'] = df['date'].dt.dayofweek
df['weekend'] = 0
df.loc[df['week7'] == 5, 'weekend'] = 1
df.loc[df['week7'] == 6, 'weekend'] = 1

df1 = df[df['station_nbr'] == 1]; df11 = df[df['station_nbr'] == 11]
df2 = df[df['station_nbr'] == 2]; df12 = df[df['station_nbr'] == 12]
df3 = df[df['station_nbr'] == 3]; df13 = df[df['station_nbr'] == 13]
df4 = df[df['station_nbr'] == 4]; df14 = df[df['station_nbr'] == 14]
df5 = df[df['station_nbr'] == 5]; df15 = df[df['station_nbr'] == 15]
df6 = df[df['station_nbr'] == 6]; df16 = df[df['station_nbr'] == 16]
df7 = df[df['station_nbr'] == 7]; df17 = df[df['station_nbr'] == 17]
df8 = df[df['station_nbr'] == 8]; df18 = df[df['station_nbr'] == 18]
df9 = df[df['station_nbr'] == 9]; df19 = df[df['station_nbr'] == 19]
df10 = df[df['station_nbr'] == 10]; df20 = df[df['station_nbr'] == 20]

In [3]:
df4 = df4.apply(pd.to_numeric, errors = 'coerce')
df4.describe().iloc[:, 14:]
# 없는 Column = codesum, station_nbr, date, store_nbr

,stnpressure,sealevel,resultspeed,resultdir,avgspeed,nothing,rain,other,store_nbr,station_nbr,item_nbr,units,week7,weekend
count,101676.000000,100344.000000,101565.000000,101565.000000,101898.000000,102120.000000,102120.000000,102120.000000,102120.0,102120.0,102120.000000,102120.000000,102120.000000,102120.000000
mean,29.322151,30.009204,3.433989,17.531148,5.149564,0.633696,0.351087,0.086957,8.0,4.0,56.000000,0.444839,3.005435,0.286957
std,0.157573,0.169471,2.398630,9.156360,2.256325,0.481796,0.477312,0.281773,0.0,0.0,32.041796,4.421178,2.000274,0.452343
min,28.870000,29.540000,0.000000,1.000000,0.500000,0.000000,0.000000,0.000000,8.0,4.0,1.000000,0.000000,0.000000,0.000000
25%,29.220000,29.900000,1.500000,14.000000,3.400000,0.000000,0.000000,0.000000,8.0,4.0,28.000000,0.000000,1.000000,0.000000
50%,29.300000,29.980000,2.900000,17.000000,4.800000,1.000000,0.000000,0.000000,8.0,4.0,56.000000,0.000000,3.000000,0.000000
75%,29.410000,30.100000,4.900000,19.000000,6.800000,1.000000,1.000000,0.000000,8.0,4.0,84.000000,0.000000,5.000000,1.000000
max,29.930000,30.650000,12.200000,36.000000,12.800000,1.000000,1.000000,1.000000,8.0,4.0,111.000000,189.000000,6.000000,1.000000


In [6]:
df4_drop_columns = ['date', 'station_nbr', 'codesum', 'store_nbr']
df4 = df4.drop(columns = df4_drop_columns)

In [5]:
df4['store_nbr'].unique()

array([8], dtype=int64)

In [7]:
# np.nan를 포함하고 있는 변수(column)를 찾아서, 그 변수에 mean 값 대입해서 Frame의 모든 Value가 fill 되게 하기.
df4_columns = df4.columns

# Cateogry 값을 포함하는 변수는 np.nan에 mode값으로 대체하고, 나머지 실수 값을 포함한 변수는 np.nan에 mean값으로 대체
for i in df4_columns:
    if (i == 'resultdir'):
        df4[i].fillna(df4[i].mode()[0], inplace=True)
        print(df4[i].mode()[0])
    else:
        df4[i].fillna(df4[i].mean(), inplace=True)

# 이제 모든 변수가 숫자로 표기 되었기 때문에, 가능 함. 
# 상대 습도 추가 #
df4['relative_humility'] = 100*(np.exp((17.625*((df4['dewpoint']-32)/1.8))/(243.04+((df4['dewpoint']-32)/1.8)))/np.exp((17.625*((df4['tavg']-32)/1.8))/(243.04+((df4['tavg']-32)/1.8))))

# 체감온도 계산
df4["windchill"] = 35.74 + 0.6215*df4["tavg"] - 35.75*(df4["avgspeed"]**0.16) + 0.4275*df4["tavg"]*(df4["avgspeed"]**0.16)

df4 = df4[df4['units'] != 0]

17.0


In [8]:
model_df4 = sm.OLS.from_formula('np.log1p(units) ~ tmax + tmin + tavg + dewpoint + wetbulb + heat + cool + preciptotal + stnpressure + \
sealevel + resultspeed + resultdir + avgspeed + C(nothing) + C(rain) + C(other) + C(item_nbr) + C(week7) + \
C(weekend) + relative_humility + windchill + 0', data = df4)
result_df4 = model_df4.fit()

print(result_df4.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.781
Model:                            OLS   Adj. R-squared:                  0.779
Method:                 Least Squares   F-statistic:                     364.1
Date:                Wed, 04 Jul 2018   Prob (F-statistic):               0.00
Time:                        13:34:09   Log-Likelihood:                -2196.0
No. Observations:                2890   AIC:                             4450.
Df Residuals:                    2861   BIC:                             4623.
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(nothing)[0]         -0.0654      0

In [9]:
anova_result_df4 = sm.stats.anova_lm(result_df4, typ=2).sort_values(by=['PR(>F)'], ascending = False)
anova_result_df4[anova_result_df4['PR(>F)'] <= 0.05]

,sum_sq,df,F,PR(>F)
preciptotal,1.202758,1.0,4.449094,3.500644e-02
C(weekend),3.028009,1.0,11.200832,8.281712e-04
C(week7),34.875524,6.0,21.501197,7.831811e-25
C(item_nbr),2687.044122,5.0,1987.915579,0.000000e+00


In [10]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(df4.values, i) for i in range(df4.shape[1])]
vif["features"] = df4.columns
vif = vif.sort_values("VIF Factor").reset_index(drop=True)
vif
# 10순위까지 겹치는것만 쓴다
# item_nbr, weekend, week7, preciptotal 

,VIF Factor,features
0,1.191320,snowfall
1,1.193694,resultdir
2,1.269742,preciptotal
3,1.414991,item_nbr
4,1.454423,units
5,1.521454,other
6,2.686089,week7
7,2.715377,weekend
8,2.876707,depart
9,5.668995,sunset


In [11]:
# item_nbr, weekend, week7, preciptotal 
model_df4 = sm.OLS.from_formula('np.log1p(units) ~ C(item_nbr) + C(week7) + C(weekend) + scale(preciptotal) + 0', data = df4)
result_df4 = model_df4.fit()

print(result_df4.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.777
Model:                            OLS   Adj. R-squared:                  0.776
Method:                 Least Squares   F-statistic:                     835.3
Date:                Wed, 04 Jul 2018   Prob (F-statistic):               0.00
Time:                        13:35:31   Log-Likelihood:                -2221.3
No. Observations:                2890   AIC:                             4469.
Df Residuals:                    2877   BIC:                             4546.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(item_nbr)[5]         3.3852      0

In [12]:
X4 = df4[['week7', 'weekend', 'item_nbr', 'preciptotal']]
y4 = df4['units']
model4 = LinearRegression()

cv4 = KFold(n_splits=10, shuffle=True, random_state=0)

cross_val_score(model4, X4, y4, scoring="r2", cv=cv4)

array([0.25420789, 0.27426891, 0.25387764, 0.3086726 , 0.29318267,
       0.32924349, 0.26420928, 0.32286589, 0.26165647, 0.3288225 ])